In [325]:
#####Switching mixture model for Access tracking####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(98537)

In [326]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [327]:
####データの発生####
##データの設定
v = 100   #コマンド数
k = 4   #混合数
m0 = 5   #通常パターンの状態数
m1 = 3   #異常パターンの状態数
hh = 3000   #ユーザー数
session = np.random.poisson(np.random.gamma(22.5, 1/0.4, hh), hh)   #セッション数
f = np.sum(session)   #総セッション数
pt = np.random.poisson(np.random.gamma(10, 1/0.55, f), f)   #セッションあたりの観測数
pt[pt < 3] = 3
hhpt = np.sum(pt)   #総観測数

In [328]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(np.arange(hh), session)
d_id = np.repeat(user_id, pt)
session_id = np.repeat(np.arange(f), pt)
no_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(f)])))

#インデックスの設定
user_list = [i for i in range(hh)]
d_list = [i for i in range(hh)]
session_list = [i for i in range(f)]
for i in range(hh):
    user_list[i] = np.array(np.where(user_id==i)[0], dtype="int")
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")
for i in range(f):
    if i==0:
        session_list[i] = np.arange(pt[i])
    else:
        session_list[i] = np.max(session_list[i-1]) + np.arange(pt[i]) + 1

In [329]:
##パラメータの設定
#潜在変数のパラメータの設定
delta = np.array([0.05])
theta = np.random.dirichlet(np.repeat(0.5, k), hh)
omega0 = np.random.dirichlet(np.repeat(1.0, m0), k)
omega1 = np.random.dirichlet(np.repeat(1.0, m1), 1).reshape(-1)
gamma0 = np.zeros((m0, m0, k)); gamma1 = np.zeros((m1, m1))
for i in range(k):
    for j in range(m0):
        gamma0[j, :, i] = np.random.dirichlet((np.full((m0, m0), 1.0) + np.diag(np.repeat(1.5, m0)))[0, ], 1).reshape(-1)
for j in range(m1):
    gamma1[j, ] = np.random.dirichlet((np.full((m1, m1), 1.0) + np.diag(np.repeat(1.5, m1)))[0, ], 1).reshape(-1)
deltat = delta.copy(); thetat = theta.copy()
omegat0 = omega0.copy(); omegat1 = omega1.copy(); gammat0 = gamma0.copy(); gammat1 = gamma1.copy()
    
#モデルパラメータの設定
phi0 = np.zeros((m0, v, k))
for j in range(k):
    phi0[:, :, j] = np.random.dirichlet(np.repeat(0.05, v), m0)
phi1 = np.random.dirichlet(np.repeat(0.05, v), m1)
phit0 = phi0.copy(); phit1 = phi1.copy()

In [330]:
##潜在変数を生成
#異常セッションを生成
A = np.random.binomial(1, delta, f)

#セッションのトピックを生成
Z = np.array(rmnom(theta[user_id, ], f, k, np.arange(f)).todense())
z_vec = np.dot(Z, np.arange(k))

##状態の系列を生成
#状態の格納用配列
S0 = np.array(np.zeros((hhpt, m0)), dtype="int")
S1 = np.array(np.zeros((hhpt, m1)), dtype="int")

#セッションごとに系列を生成
for i in range(f):
    #データの設定
    z = z_vec[i]
    index = session_list[i]

    #異常状態の系列を生成
    if A[i]==1:
        for j in range(pt[i]):
            index_record = index[j]
            if j==0:
                S1[index_record, ] = np.random.multinomial(1, omega1, 1)
            else:
                gamma = gamma1[np.argmax(S1[index_record-1, ]), ]
                S1[index_record, ] = np.random.multinomial(1, gamma, 1)
    else:

        #通常状態の系列を生成
        for j in range(pt[i]):
            index_record = index[j]
            if j==0:
                omega = omega0[z, ]
                S0[index_record, ] = np.random.multinomial(1, omega, 1)
            else:
                gamma = gamma0[np.argmax(S0[index_record-1, ]), :, z]
                S0[index_record, ] = np.random.multinomial(1, gamma, 1)

#系列をベクトルに変換
a_vec = A[session_id]
s0 = (1-a_vec) * (np.dot(S0, np.arange(m0)) + 1)
s1 = a_vec * (np.dot(S1, np.arange(m1)) + 1)

In [331]:
##系列の観測変数を生成
#インデックスの設定
index_a0 = np.array(np.where(a_vec==0)[0], dtype="int")
index_a1 = np.delete(np.arange(hhpt), index_a0)
n0 = index_a0.shape[0]; n1 = index_a1.shape[0]

#多項分布から観測変数を生成
y = np.array(np.zeros((hhpt, v)), dtype="int")
y[index_a0, ] = np.array(rmnom(phi0[s0[index_a0]-1,: ,z_vec[session_id][index_a0]], n0, v, np.arange(n0)).todense(), dtype="int")
y[index_a1, ] = np.array(rmnom(phi1[s1[index_a1]-1, ], n1, v, np.arange(n1)).todense(), dtype="int")
y_vec = np.dot(y, np.arange(v))
sparse_data = sparse.csr_matrix(y)
sparse_data_T = sparse_data.T

In [431]:
####MCMCでパラメータを推定####
##アルゴリズムの設定
R = 2000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = int(500/keep)

In [432]:
##事前分布の設定
#潜在変数の事前分布
a0 = 1; b0 = 1
alpha0 = 0.1
beta0 = 1.0

#モデルパラメータの事前分布
alpha01 = 0.1
alpha02 = 0.1

In [433]:
##系列のインデックスの設定
#先頭と後尾のインデックスを作成
max_no = np.max(no_id) + 1
index_t1 = np.array(np.where(no_id==0)[0], dtype="int")
index_t12 = np.repeat(0, f)
for i in range(hh):
    index_t12[i] = np.max(session_list[i])
    
#中間のインデックスを作成
index_list_t21 = [j for j in range(max_no-1)]
index_list_t22 = [j for j in range(max_no-1)]
for j in range(1, max_no):
    index_list_t21[j-1] = np.array(np.where(no_id==j)[0], dtype="int") - 1
    index_list_t22[j-1] = np.array(np.where(no_id==j)[0], dtype="int")
index_t21 = np.sort(np.array(list(itertools.chain(*[index_list_t21[j] for j in range(max_no-1)]))))
index_t22 = np.sort(np.array(list(itertools.chain(*[index_list_t22[j] for j in range(max_no-1)]))))

In [434]:
##データとインデックスの設定
#データ設定
d_dt = sparse.coo_matrix((np.repeat(1, f), (user_id, np.arange(f))), shape=(hh, f)).tocsr()

#インデックスの設定
v_list = [j for j in range(v)]
v_vec = [j for j in range(v)]
for j in range(v):
    v_list[j] = np.array(np.where(y_vec==j)[0], dtype="int")
    v_vec[j] = np.repeat(1, v_list[j].shape[0])

In [435]:
##パラメータの真値
#潜在変数のパラメータの真値
delta = delta.copy()
theta = thetat.copy()
omega0 = omegat0.copy(); omega1 = omegat1.copy()
gamma0 = gammat0.copy(); gammat1 = gammat1.copy()
    
#モデルパラメータの真値
phi0 = phit0.copy()
phi1 = phit1.copy()

#潜在状態の真値
Ai = A.copy()
Zi = Z.copy()
z_vec = np.dot(Zi, np.arange(k))
Si0 = S0.copy()
Si1 = S1.copy()

In [ ]:
##パラメータの初期値
#潜在変数のパラメータの初期値
delta = np.array([0.1])
theta = np.random.dirichlet(np.repeat(5.0, k), hh)
omega0 = np.random.dirichlet(np.repeat(5.0, m0), k)
omega1 = np.random.dirichlet(np.repeat(5.0, m1), 1).reshape(-1)
gamma0 = np.zeros((m0, m0, k)); gamma1 = np.zeros((m1, m1))
for i in range(k):
    for j in range(m0):
        gamma0[j, :, i] = np.random.dirichlet((np.full((m0, m0), 5.0) + np.diag(np.repeat(5.0, m0)))[0, ], 1).reshape(-1)
for j in range(m1):
    gamma1[j, ] = np.random.dirichlet((np.full((m1, m1), 5.0) + np.diag(np.repeat(5.0, m1)))[0, ], 1).reshape(-1)
    
#モデルパラメータの初期値
phi0 = np.zeros((m0, v, k))
for j in range(k):
    phi0[:, :, j] = np.random.dirichlet(np.repeat(5.0, v), m0)
phi1 = np.random.dirichlet(np.repeat(5.0, v), m1)

In [ ]:
##潜在変数を初期値
#異常セッションを生成
A = np.random.binomial(1, delta, f)

#セッションのトピックを生成
Z = np.array(rmnom(theta[user_id, ], f, k, np.arange(f)).todense())
z_vec = np.dot(Z, np.arange(k))

##状態の系列を生成
#状態の格納用配列
S0 = np.array(np.zeros((hhpt, m0)), dtype="int")
S1 = np.array(np.zeros((hhpt, m1)), dtype="int")

#セッションごとに系列を生成
for i in range(f):
    #データの設定
    z = z_vec[i]
    index = session_list[i]

    #異常状態の系列を生成
    if A[i]==1:
        for j in range(pt[i]):
            index_record = index[j]
            if j==0:
                S1[index_record, ] = np.random.multinomial(1, omega1, 1)
            else:
                gamma = gamma1[np.argmax(S1[index_record-1, ]), ]
                S1[index_record, ] = np.random.multinomial(1, gamma, 1)
    else:

        #通常状態の系列を生成
        for j in range(pt[i]):
            index_record = index[j]
            if j==0:
                omega = omega0[z, ]
                S0[index_record, ] = np.random.multinomial(1, omega, 1)
            else:
                gamma = gamma0[np.argmax(S0[index_record-1, ]), :, z]
                S0[index_record, ] = np.random.multinomial(1, gamma, 1)

#系列をベクトルに変換
a_vec = A[session_id]
s0 = (1-a_vec) * (np.dot(S0, np.arange(m0)) + 1)
s1 = a_vec * (np.dot(S1, np.arange(m1)) + 1)

In [436]:
##すべての潜在状態での系列を生成
#状態の格納用配列
vec_k = np.arange(k); vec_m0 = np.arange(m0)
new_S0 = np.array(np.zeros((hhpt, m0, k)), dtype="int")
new_S1 = np.array(np.zeros((hhpt, m1)), dtype="int")

#セッションごとの系列を生成
for i in range(max_no):
    #1回目の系列を生成
    if i==0:
        index = index_t1.copy()
        n = index.shape[0]
        for j in range(k):
            new_S0[index, :, j] = np.random.multinomial(1, omega0[j, ], n)
        new_S1[index, ] = np.random.multinomial(1, omega1, n)
        
    else:
        
        #2回目以降の系列を生成
        index1 = index_list_t21[i-1]
        index2 = index_list_t22[i-1]
        n = index1.shape[0]
        for j in range(k):
            z0 = np.dot(new_S0[index1, :, j], np.arange(m0))
            new_S0[index2, :, j] = np.array(rmnom(gamma0[:, :, j][z0, ], n, m0, np.arange(n)).todense())
        z1 = np.dot(new_S1[index1, ], np.arange(m1))
        new_S1[index2, ] = np.array(rmnom(gamma1[z1, ], n, m1, np.arange(n)).todense())
        
#系列をベクトルに変換
new_s0 = np.array(np.zeros((hhpt, k)), dtype="int")
for j in range(k):
    new_s0[:, j] = np.dot(new_S0[:, :, j], np.arange(m0))
new_s1 = np.dot(new_S1, np.arange(m1))

In [438]:
##真値での対数尤度
#潜在状態を条件づけた対数尤度
a_vec = A[session_id]
s0 = (1-a_vec) * (np.dot(S0, np.arange(m0)) + 1)
s1 = a_vec * (np.dot(S1, np.arange(m1)) + 1)
index_s0 = np.array(np.where(s0 > 0)[0], dtype="int")
index_s1 = np.array(np.where(s1==1)[0], dtype="int")
LLi0 = np.repeat(0.0, index_s0.shape[0])
for j in range(k):
    LLi0 += np.dot((Z[session_id, j][:, np.newaxis] * S0 * (phit0[:, :, j].T)[y_vec, ])[index_s0, ], np.repeat(1, m0))
LLi1 = np.dot((A[session_id][:, np.newaxis] * S1 * (phit1.T)[y_vec, ])[index_s1, ], np.repeat(1, m1))

#対数尤度の和
LLbest0 = np.sum(np.log(LLi0)) 
LLbest1 = np.sum(np.log(LLi1))
LLbest = LLbest0 + LLbest1
print(LLbest)

-6532532.287873241


In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##潜在状態ごとの事後分布を設定
    #潜在状態ごとの尤度
    Lho0 = np.zeros((hhpt, m0, k))
    for j in range(k):
        Lho0[:, :, j] = (phi0[:, :, j].T)[y_vec, ]
    Lho1 = (phi1.T)[y_vec, ]

    #すべての潜在状態の事前確率
    gamma_dt01 = np.full((hhpt, m0, k), 1/m0); gamma_dt02 = gamma_dt01.copy()
    gamma_dt11 = np.full((hhpt, m1), 1/m1); gamma_dt12 = gamma_dt11.copy()
    for j in range(k):
        gamma_dt01[index_t1, :, j] = np.full((f, m0), omega0[j, ]); gamma_dt01[index_t22, :, j] = gamma0[new_s0[index_t21, j], :, j]
        gamma_dt02[index_t21, :, j] =(gamma0[:, :, j].T)[new_s0[index_t22, j], ]
    gamma_dt11[index_t1, ] = np.full((f, m1), omega1); gamma_dt11[index_t22, ] = gamma1[new_s1[index_t21], ]
    gamma_dt12[index_t21, ] = (gamma1.T)[new_s1[index_t22], ]

    #潜在状態ごとの事後分布
    Posterior0 = gamma_dt01 * gamma_dt02 * Lho0
    Posterior1 = gamma_dt11 * gamma_dt12 * Lho1
    Posterior_mu0 = np.zeros((hhpt, k))
    for j in range(k):
        Posterior_mu0[:, j] = np.dot(Posterior0[:, :, j], np.repeat(1, m0))
    Posterior_mu1 = np.dot(Posterior1, np.repeat(1, m1))
    Posterior_mu = np.hstack((Posterior_mu0, Posterior_mu1[:, np.newaxis]))
    Posterior_log = np.log(Posterior_mu) 

    #事後分布が桁落ちしないように変換
    LLho = np.zeros((f, k+1))
    for i in range(f):
        LLho[i, ] = np.sum(Posterior_log[session_list[i], ], axis=0) 
    LLho_exp = np.exp(LLho - np.max(LLho, axis=1)[:, np.newaxis])


    ##セッションの異常状態の切換変数を生成
    #異常状態のセッションの割当確率
    Topic_par = theta[user_id, ] * LLho_exp[:, :k]
    Delta = np.full((f, 2), np.append(1-delta, delta))
    delta_allocation = Delta * np.hstack((np.dot(Topic_par, np.repeat(1, k))[:, np.newaxis], LLho_exp[:, k][:, np.newaxis]))
    Prob = delta_allocation / np.sum(delta_allocation, axis=1)[:, np.newaxis]

    #ベルヌーイ分布から切換変数を生成
    Ai = np.random.binomial(1, Prob[:, 1], f)
    index_a = np.array(np.where(Ai==1)[0], dtype="int")

    ##セッションのトピックを生成
    #トピックの割当確率
    Prob = Topic_par / np.dot(Topic_par, np.repeat(1, k))[:, np.newaxis]

    #多項分布からトピックを生成
    Zi_sparse = rmnom(Prob, f, k, np.arange(f))
    Zi = (1-Ai[:, np.newaxis]) * np.array(Zi_sparse.todense())
    G = np.hstack((Zi, Ai[:, np.newaxis]))


    ##潜在状態の系列を生成
    #すべての潜在状態での系列を生成
    for j in range(k):
        Prob0 = Posterior0[:, :, j] / np.dot(Posterior0[:, :, j], np.repeat(1, m0))[:, np.newaxis]
        new_S0[:, :, j] = np.array(rmnom(Prob0, hhpt, m0, np.arange(hhpt)).todense())
        new_s0[:, j] = np.dot(new_S0[:, :, j], np.arange(m0))
    Prob1 = Posterior1 / np.dot(Posterior1, np.repeat(1, m1))[:, np.newaxis]
    new_S1 = np.array(rmnom(Prob1, hhpt, m1, np.arange(hhpt)).todense())
    new_s1 = np.dot(new_S1, np.arange(m1))

    #潜在状態に応じて新しい系列を生成
    Zi_dt = Zi[session_id, ]
    Si0 = np.array(np.zeros((hhpt, m0)), dtype="int")
    a_vec = Ai[session_id]
    for j in range(k):
        Si0 += Zi_dt[:, j][:, np.newaxis] * new_S0[:, :, j]
    Si1 = a_vec[:, np.newaxis] * new_S1

    #系列をベクトルに変換
    s0 = (1-a_vec) * (np.dot(S0, np.arange(m0)) + 1)
    s1 = a_vec * (np.dot(S1, np.arange(m1)) + 1)
    
    
    ##潜在変数の混合率をサンプリング
    #異常状態のセッションの混合率をサンプリング
    af = np.sum(Ai)
    a = af + a0
    b = f - af + b0
    delta = np.random.beta(a, b, 1)

    #トピック分布をサンプリング
    index_a0 = np.delete(np.arange(f), index_a)
    wf = np.array(np.dot(d_dt[:, index_a0], Zi_sparse.tocsc()[index_a0, ]).todense()) + alpha0
    for i in range(hh):
        theta[i, ] = np.random.dirichlet(wf[i, ], 1)

    #潜在状態の初期確率をサンプリング
    new_Si0 = np.zeros((hhpt, m0, k))
    rf01 = np.array(np.zeros((k, m0)), dtype="int")
    for j in range(k):
        new_Si0[:, :, j] = Zi_dt[:, j][:, np.newaxis] * Si0
        rf01[j, ] = np.sum(new_Si0[index_t1, :, j], axis=0) + beta0
        omega0[j, ] = np.random.dirichlet(rf01[j, ], 1)
    rf11 = np.sum(Si1[index_t1, ], axis=0) + beta0
    omega1 = np.random.dirichlet(rf11, 1).reshape(-1)

    #潜在状態の推移確率をサンプリング
    rf02 = np.array(np.zeros((m0, m0, k)))
    for j1 in range(k):
        rf02[:, :, j1] = np.dot(new_Si0[index_t21, :, j1].T, new_Si0[index_t22, :, j1]) + beta0
        for j2 in range(m0):
            gamma0[j2, :, j1] = np.random.dirichlet(rf02[j2, :, j1], 1)
    rf12 = np.dot(Si1[index_t21, ].T, Si1[index_t22, ]) + beta0
    for j in range(m1):
        gamma1[j, ] = np.random.dirichlet(rf12[j, ], 1)


    ##モデルパラメータをサンプリング
    #通常状態のモデルパラメータをサンプリング
    vf0 = np.zeros((m0, v, k))
    for j1 in range(k):
        for j2 in range(v):
            vf0[:, j2, j1] = np.dot(new_Si0[v_list[j2], :, j1].T, v_vec[j2]) + alpha01
        for j3 in range(m0):
            phi0[j3, :, j1] = np.random.dirichlet(vf0[j3, :, j1], 1)

    #異常状態のモデルパラメータをサンプリング
    vf1 = np.zeros((m1, v))
    for j in range(v):
        vf1[:, j] = np.dot(Si1[v_list[j], ].T, v_vec[j]) + alpha02
    for j in range(m1):
        phi1[j, ] = np.random.dirichlet(vf1[j, ], 1)
        

    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep

    #対数尤度を更新
    index_s0 = np.array(np.where(s0 > 0)[0], dtype="int"); index_s1 = np.array(np.where(s1==1)[0], dtype="int")
    LLi0 = np.repeat(0.0, index_s0.shape[0])
    for j in range(k):
        LLi0 += np.dot((Zi[session_id, j][:, np.newaxis] * Si0 * Lho0[:, :, j])[index_s0, ], np.repeat(1, m0))
    LLi1 = np.dot((Ai[session_id][:, np.newaxis] * Si1 * Lho1)[index_s1, ], np.repeat(1, m1))

    #対数尤度の和
    LL0 = np.sum(np.log(LLi0)) 
    LL1 = np.sum(np.log(LLi1))
    LL = LL0 + LL1

    if rp%disp==0:
        #サンプリング結果を確認
        print(rp)
        print(np.round(np.array([LL, LLbest]), 1))

0
[-6891963.9 -6532532.3]
10
[-6501562.4 -6532532.3]
20
[-6480682.6 -6532532.3]
30
[-6470675.9 -6532532.3]
40
[-6464941.  -6532532.3]
50
[-6459264.2 -6532532.3]
60
[-6458994.5 -6532532.3]
70
[-6457456.8 -6532532.3]
80
[-6451994.5 -6532532.3]
90
[-6453785.  -6532532.3]
100
[-6456169.6 -6532532.3]
110
[-6453259.7 -6532532.3]
